In [1]:
from mindspore import context
from mindspore.nn import Adam
from mindsponge import Sponge
from mindsponge import Molecule
from mindsponge import ForceField
from mindsponge import DynamicUpdater
from mindsponge import SimulationCell
from mindsponge.control import VelocityVerlet, Langevin
from mindsponge.potential import SphericalRestrict
from mindsponge.function import VelocityGenerator
from mindsponge.callback import WriteH5MD, RunInfo

In [2]:
context.set_context(mode=context.GRAPH_MODE, device_target="GPU")

In [3]:
system = Molecule(template='water.spce.yaml')

In [4]:
system.reduplicate([0.3, 0, 0])
system.reduplicate([0, 0.3, 0])
new_sys = system.copy([0, 0, -0.3])
system.reduplicate([0, 0, 0.3])
system.append(new_sys)

Molecule<>

In [5]:
potential = ForceField(system, parameters='SPCE')

In [6]:
opt = Adam(system.trainable_params(), 1e-3)

In [7]:
mini = Sponge(system, potential, opt)

TotalTime = 0.3938, [19]
[parse]: 0.0135589
[symbol_resolve]: 0.07029, [1]
    [Cycle 1]: 0.0698824, [1]
        [resolve]: 0.0698705
[combine_like_graphs]: 2.53506e-06
[inference_opt_prepare]: 0.000402823
[abstract_specialize]: 0.267594
[auto_monad]: 0.000893295
[inline]: 7.79144e-06
[py_pre_ad]: 1.42306e-06
[pynative_shard]: 2.95602e-06
[pipeline_split]: 6.31995e-06
[optimize]: 0.0296914, [15]
    [simplify_data_structures]: 0.00041726
    [opt_a]: 0.0280925, [2]
        [Cycle 1]: 0.0268281, [25]
            [expand_dump_flag]: 1.4089e-05
            [switch_simplify]: 0.000839872
            [a_1]: 0.00846185
            [recompute_prepare]: 2.33185e-05
            [updatestate_depend_eliminate]: 2.67774e-05
            [updatestate_assign_eliminate]: 2.24002e-05
            [updatestate_loads_eliminate]: 3.00836e-05
            [parameter_eliminate]: 3.49991e-06
            [a_2]: 0.000876825
            [accelerated_algorithm]: 1.83415e-05
            [auto_parallel]: 6.12997e-06

In [8]:
run_info = RunInfo(10)
mini.run(1000, callbacks=[run_info])

TotalTime = 0.0137587, [19]
[parse]: 0.00441828
[symbol_resolve]: 0.0011176, [1]
    [Cycle 1]: 0.00108488, [1]
        [resolve]: 0.00107912
[combine_like_graphs]: 6.22123e-07
[inference_opt_prepare]: 3.10205e-05
[abstract_specialize]: 0.0006884
[auto_monad]: 0.000110507
[inline]: 1.99676e-06
[py_pre_ad]: 1.27405e-06
[pynative_shard]: 1.68569e-06
[pipeline_split]: 2.13832e-06
[optimize]: 0.00190804, [15]
    [simplify_data_structures]: 8.10064e-06
    [opt_a]: 0.00155111, [2]
        [Cycle 1]: 0.000898985, [25]
            [expand_dump_flag]: 1.61491e-06
            [switch_simplify]: 1.02054e-05
            [a_1]: 0.000158457
            [recompute_prepare]: 3.64333e-06
            [updatestate_depend_eliminate]: 1.78702e-05
            [updatestate_assign_eliminate]: 3.89852e-06
            [updatestate_loads_eliminate]: 4.17605e-06
            [parameter_eliminate]: 1.68942e-06
            [a_2]: 6.38925e-05
            [accelerated_algorithm]: 6.94767e-06
            [auto_parall

In [9]:
temp = 300
vgen = VelocityGenerator(temp)
velocity = vgen(system.coordinate.shape, system.atom_mass)

TotalTime = 0.121899, [19]
[parse]: 0.00990961
[symbol_resolve]: 0.0465803, [1]
    [Cycle 1]: 0.0456885, [1]
        [resolve]: 0.0456769
[combine_like_graphs]: 9.2946e-07
[inference_opt_prepare]: 0.000445476
[abstract_specialize]: 0.0473313
[auto_monad]: 0.000320559
[inline]: 2.11783e-06
[py_pre_ad]: 2.04705e-06
[pynative_shard]: 1.37091e-06
[pipeline_split]: 4.01028e-06
[optimize]: 0.00781896, [15]
    [simplify_data_structures]: 0.000142027
    [opt_a]: 0.00707382, [2]
        [Cycle 1]: 0.00617015, [25]
            [expand_dump_flag]: 6.82659e-06
            [switch_simplify]: 0.000289891
            [a_1]: 0.0019913
            [recompute_prepare]: 7.5791e-06
            [updatestate_depend_eliminate]: 8.6613e-06
            [updatestate_assign_eliminate]: 7.21402e-06
            [updatestate_loads_eliminate]: 7.36862e-06
            [parameter_eliminate]: 2.07499e-06
            [a_2]: 0.000286831
            [accelerated_algorithm]: 7.17863e-06
            [auto_parallel]: 3.71

In [10]:
opt = DynamicUpdater(system,
                     integrator=VelocityVerlet(system),
                     thermostat=Langevin(system, temp),
                     velocity=velocity,
                     time_step=5e-4)

## 外加限制

在MindSponge的SimulationCell中可以自定义一些外加的势能项，这是一个非常灵活的操作，支持同时定义多个Bias叠加使用，并且在MindSpore框架的支持下都是可微的。比如本案例中施加的Bias是中心球形势SphericalRestrict，用于把原子的运动限制在一个球形范围内。另外MindSponge也支持原位谐振子势OscillatorBias，可以将原子限制在原始位置附近，这个功能在蛋白质预测中能够起到较大的作用。基于MindSponge的基础类PotentialCell，用户还可以自定义更多种类的Bias。

In [11]:
sim = SimulationCell(system, potential, bias=SphericalRestrict(radius=1.5, center=[0, 0, 0]))
md = Sponge(sim, optimizer=opt)

TotalTime = 0.326642, [19]
[parse]: 0.00283996
[symbol_resolve]: 0.0457982, [1]
    [Cycle 1]: 0.0451715, [1]
        [resolve]: 0.0451634
[combine_like_graphs]: 8.08388e-07
[inference_opt_prepare]: 0.000411255
[abstract_specialize]: 0.228701
[auto_monad]: 0.000989549
[inline]: 3.36394e-06
[py_pre_ad]: 2.08057e-06
[pynative_shard]: 1.44728e-06
[pipeline_split]: 4.82239e-06
[optimize]: 0.0366946, [15]
    [simplify_data_structures]: 0.000436593
    [opt_a]: 0.0351406, [2]
        [Cycle 1]: 0.0338761, [25]
            [expand_dump_flag]: 1.78497e-05
            [switch_simplify]: 0.000874396
            [a_1]: 0.00873139
            [recompute_prepare]: 2.36835e-05
            [updatestate_depend_eliminate]: 2.24877e-05
            [updatestate_assign_eliminate]: 1.99601e-05
            [updatestate_loads_eliminate]: 1.96267e-05
            [parameter_eliminate]: 2.0992e-06
            [a_2]: 0.000879327
            [accelerated_algorithm]: 1.5568e-05
            [auto_parallel]: 3.7793

In [12]:
cb_h5md = WriteH5MD(system, 'tutorial_b04.h5md', save_freq=10, write_velocity=True, write_force=True)

In [13]:
md.run(3000, callbacks=[run_info, cb_h5md])

TotalTime = 0.328543, [19]
[parse]: 0.00229323
[symbol_resolve]: 0.0494214, [1]
    [Cycle 1]: 0.0484167, [1]
        [resolve]: 0.0484033
[combine_like_graphs]: 9.89065e-07
[inference_opt_prepare]: 0.000467516
[abstract_specialize]: 0.231098
[auto_monad]: 0.00105964
[inline]: 2.61143e-06
[py_pre_ad]: 1.6056e-06
[pynative_shard]: 1.59629e-06
[pipeline_split]: 4.1686e-06
[optimize]: 0.0322995, [15]
    [simplify_data_structures]: 0.000440734
    [opt_a]: 0.030631, [2]
        [Cycle 1]: 0.029219, [25]
            [expand_dump_flag]: 2.87481e-05
            [switch_simplify]: 0.000861553
            [a_1]: 0.00905391
            [recompute_prepare]: 2.90927e-05
            [updatestate_depend_eliminate]: 8.25915e-05
            [updatestate_assign_eliminate]: 4.70448e-05
            [updatestate_loads_eliminate]: 2.49501e-05
            [parameter_eliminate]: 1.83471e-06
            [a_2]: 0.00101082
            [accelerated_algorithm]: 1.84271e-05
            [auto_parallel]: 3.52412e-0